# Laboratory 6

## Exercise 5
The number of particles emitted by a radioactive source during a fixed interval of time ($\Delta t = 10\text{s}$) follows a Poisson distribution on the parameter µ. The number of particles observed during consecutive time intervals is: $4, 1, 3, 1$ and $3$.

### a) Suppose a uniform prior distribution for the parameter $\mu$ 
- determine and draw the posterior distribution for $\mu$, given the data
- evaluate mean, median and variance, both analytically and numerically in R

In [2]:
normalized <- function(f,a,b){
    c <- integrate(f,a,b)$value
    n_f <- function(x){f(x)/c}
    return(n_f)
}

prior <- function(x){1}
# The product of poisson distributions becomes a gamma function:
likelihood <- function(data,x){dgamma(x,sum(data)+1,length(data))}

posterior <- function(prior,likelihood)
    
}


In [6]:
normalized

ERROR: Error in eval(expr, envir, enclos): object 'normalized' not found
